In [66]:
!pip install pyTelegramBotAPI
!pip3 install python-telegram-bot
import os
from glob import glob
import telebot
from telebot import types
from telegram import Update
from telegram import InlineKeyboardButton
from telegram import InlineKeyboardMarkup
from telegram.ext import CallbackContext
from telegram.ext import CallbackQueryHandler
from telegram.ext import Updater
from telegram.ext import MessageHandler
from telegram.ext import Filters
from torchvision.utils import save_image
from random import choice
from db import *
!pip3 install torch torchvision
!pip3 install pillow 7.0.0
%matplotlib inline
from PIL import Image


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt


import torchvision.transforms as transforms
import torchvision.models as models

import copy



cnn = models.vgg19(pretrained=True).features.to(device).eval()
path_style = ''

ERROR: Could not find a version that satisfies the requirement 7.0.0 (from versions: none)
ERROR: No matching distribution found for 7.0.0


In [65]:

###############################################################


TOKEN = '1715580463:AAGSuz7c8EKO43wt_0w6-Yfct8vcOfkVO6U'
PATH_GOOGLE = 'drive/MyDrive/'
PATH_SAVED_PICS = PATH_GOOGLE + 'images_download_from_users'
PATH_GEN_PICS = PATH_GOOGLE + 'images'
ANIME = PATH_GOOGLE + 'anime'     
WEBM = PATH_GOOGLE + 'webm'
photo_name = ''
bot = telebot.TeleBot(TOKEN)
key_name = 'transfer_learn'

markup = types.ReplyKeyboardMarkup(row_width=2)
itembtn1 = types.KeyboardButton('save pic')
itembtn2 = types.KeyboardButton('rand pic')
itembtn3 = types.KeyboardButton('rand webm')
itembtn4 = types.KeyboardButton('/help')
itembtn5 = types.KeyboardButton('count')
itembtn6 = types.KeyboardButton('list')
itembtn7 = types.KeyboardButton(key_name)
markup.add(itembtn1, itembtn2, itembtn3, itembtn4, itembtn7)

init_db()
if os.path.exists( 'images_for_users'):
        pass
else:
        os.mkdir( 'images_for_users')

if os.path.exists('PATH_SAVED_PICS'):
        pass
else:
        os.mkdir('PATH_SAVED_PICS')

if os.path.exists( 'images_from_users'):
        pass
else:
        os.mkdir('images_from_users')
 


def picturing(message, path):
    pic_list = glob(path + '/*')
    if message.text.isdigit() :
        if 1 <= int(message.text) <= len(pic_list):
            picture = pic_list[int(message.text) - 1]
            if os.path.exists(picture):
                photo = open(picture, 'rb')
                bot.send_photo(message.from_user.id, photo)
            else:
                bot.send_message(message.from_user.id, 'Ты ввел некорректный номер')
        else:
            bot.send_message(message.from_user.id, 'Ты ввел некорректный номер')
    else :
        bot.send_message(message.from_user.id, 'Ты ввел некорректный номер')


def my_pic(message):
    picturing(message, 'PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}')

def pic(message):
    picturing(message, PATH_GEN_PICS)


#принимает первое фото

def photo_obj(message):
    if os.path.exists('images_for_users/{' + str(message.from_user.id) + '}'):
        directory = 'images_for_users/{' + str(message.from_user.id) + '}'
    else:
        os.mkdir('images_for_users/{' + str(message.from_user.id) + '}')
        directory = 'images_for_users/{' + str(message.from_user.id) + '}'
    if message.photo:
        photo = message.photo[-1]
        file_id = photo.file_id
        file_path = bot.get_file(file_id).file_path
        downloaded_file = bot.download_file(file_path)
        name_obj = file_id + '.jpg'
        
        new_file = open(directory + '/' + name_obj, mode='wb')
        new_file.write(downloaded_file)
        global path_style
        path_style = directory + '/' + name_obj
       

        new_file.close()
        bot.send_message(message.from_user.id, 'Теперь пришли фото, которое мы будем менять')
        bot.register_next_step_handler(message, photo_ex, name_obj)


#принимает второе фото и отправляет первое

def photo_ex(message, name_obj):
    if os.path.exists('images_for_users/{' + str(message.from_user.id) + '}'):
        directory = 'images_for_users/{' + str(message.from_user.id) + '}'
    else:
        os.mkdir('images_for_users/{' + str(message.from_user.id) + '}')
        directory = 'images_for_users/{' + str(message.from_user.id) + '}'
    if message.photo:
        photo = message.photo[-1]
        file_id = photo.file_id
        file_path = bot.get_file(file_id).file_path
        downloaded_file = bot.download_file(file_path)
        name = file_id + ".jpg"
        new_file = open(directory + '/' + name, mode='wb')
        new_file.write(downloaded_file)
        new_file.close()


        imsize = 128  

        loader = transforms.Compose([
            transforms.Resize(imsize),  # нормируем размер изображения
            transforms.CenterCrop(imsize),
            transforms.ToTensor()])  # превращаем в удобный формат
            
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        def image_loader(image_name):
            image = Image.open(image_name)
            
            image = loader(image)
            image = image.unsqueeze(0)# для удобства добавляем 1 размерность
            return image.to(device, torch.float)
            ################################
        style_img = image_loader(path_style)
        content_img = image_loader(directory + '/' + name)
        unloader = transforms.ToPILImage() # тензор в кратинку  
        def imshow(tensor, title=None):
            image = tensor  
            image = image.squeeze(0)      # функция для отрисовки изображения
            image = unloader(image)
            plt.imshow(image)
            if title is not None:
                plt.title(title)

        aa = style_img[0]
        save_image(aa, 'xui.jpg')
        class ContentLoss(nn.Module):

                def __init__(self, target,):
                    super(ContentLoss, self).__init__()
                    self.target = target.detach()#это константа. Убираем ее из дерева вычеслений
                    #self.loss = F.mse_loss(self.target, self.target )

                def forward(self, input):
                    self.loss = F.mse_loss(input, self.target)
                    return input
        def gram_matrix(input):
                batch_size , h, w, f_map_num = input.size()  # batch size(=1)
                
                features = input.view(batch_size * h, w * f_map_num)  

                G = torch.mm(features, features.t())  

                
                return G.div(batch_size * h * w * f_map_num)# normaliziruem dannye 
        class StyleLoss(nn.Module):
                def __init__(self, target_feature):
                    super(StyleLoss, self).__init__()
                    self.target = gram_matrix(target_feature).detach()
                    #self.loss = F.mse_loss(self.target, self.target)# to initialize with something

                def forward(self, input):
                    G = gram_matrix(input)
                    self.loss = F.mse_loss(G, self.target)
                    return input
        cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
        cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)
        #standartnaya normirovka dla vgg
        class Normalization(nn.Module):
                def __init__(self, mean, std):
                    super(Normalization, self).__init__()
                    # razvorachivaem v vector [C x 1 x 1]
                    
                    self.mean = torch.tensor(mean).view(-1, 1, 1)
                    self.std = torch.tensor(std).view(-1, 1, 1)

                def forward(self, img):
                    
                    return (img - self.mean) / self.std
        content_layers_default = ['conv_4']
        style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                                   style_img, content_img,
                                   content_layers=content_layers_default,
                                   style_layers=style_layers_default):
            cnn = copy.deepcopy(cnn)

            # normalizuem 
            normalization = Normalization(normalization_mean, normalization_std).to(device)

            
            # list iz losses
            content_losses = []
            style_losses = [] 

            # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
            # to put in modules that are supposed to be activated sequentially
            model = nn.Sequential(normalization)

            i = 0  # increment every time we see a conv
            for layer in cnn.children():
                if isinstance(layer, nn.Conv2d):
                    i += 1
                    name = 'conv_{}'.format(i)
                elif isinstance(layer, nn.ReLU):
                    name = 'relu_{}'.format(i)
                    # The in-place version doesn't play very nicely with the ContentLoss
                    # and StyleLoss we insert below. So we replace with out-of-place
                    # ones here.
                    #Переопределим relu уровень
                    layer = nn.ReLU(inplace=False)
                elif isinstance(layer, nn.MaxPool2d):
                    name = 'pool_{}'.format(i)
                elif isinstance(layer, nn.BatchNorm2d):
                    name = 'bn_{}'.format(i)
                else:
                    raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

                model.add_module(name, layer)

                if name in content_layers:
                    # add content loss:
                    target = model(content_img).detach()
                    content_loss = ContentLoss(target)
                    model.add_module("content_loss_{}".format(i), content_loss)
                    content_losses.append(content_loss)

                if name in style_layers:
                    # add style loss:
                    target_feature = model(style_img).detach()
                    style_loss = StyleLoss(target_feature)
                    model.add_module("style_loss_{}".format(i), style_loss)
                    style_losses.append(style_loss)

            # now we trim off the layers after the last content and style losses
            #выбрасываем все уровни после последенего styel loss или content loss
            for i in range(len(model) - 1, -1, -1):
                if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                    break# находим какой-то и, который является контенлосом или стайллосом

            model = model[:(i + 1)]# рассматриваем модель которая не включает в себя ни одного контент или стайл лоса

            return model, style_losses, content_losses

        
        def get_input_optimizer(input_img):
                #добоваляет содержимое тензора катринки в список изменяемых оптимизатором параметров
                optimizer = optim.LBFGS([input_img.requires_grad_()]) 
                return optimizer
        def run_style_transfer(cnn, normalization_mean, normalization_std,
                                content_img, style_img, input_img, num_steps=500,
                                style_weight=10000, content_weight=1):
                print('Building the style transfer model..')
                model, style_losses, content_losses = get_style_model_and_losses(cnn,
                    normalization_mean, normalization_std, style_img, content_img)
                optimizer = get_input_optimizer(input_img)

                print('Optimizing..')
                run = [0]
                while run[0] <= num_steps:

                    def closure():
                        # correct the values 
                        # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
                        input_img.data.clamp_(0, 1)

                        optimizer.zero_grad()

                        model(input_img)

                        style_score = 0
                        content_score = 0

                        for sl in style_losses:
                            style_score += sl.loss
                        for cl in content_losses:
                            content_score += cl.loss
                        
                        #взвешивание ощибки
                        style_score *= style_weight
                        content_score *= content_weight

                        loss = style_score + content_score
                        loss.backward()
                        
                        run[0] += 1
                        if run[0] %50 == 0:
                            print("numbepoch {}".format(run[0]))
                            print ("style {:4f}".format(style_score))
                            print("content {:4f}".format(content_score))
                            print()
                        return style_score + content_score

                    optimizer.step(closure)

                # a last correction...
                input_img.data.clamp_(0, 1)

                return input_img


        input_img = content_img.clone()
        # if you want to use white noise instead uncomment the below line:
        # input_img = torch.randn(content_img.data.size(), device=device)

        # add the original input image to the figure:
        plt.figure()
        imshow(input_img, title='Input Image')
        output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                    content_img, style_img, input_img)
        aa = output[0]
        save_image(aa, "pict.jpg")

        
        picture ='images_for_users/{' + str(message.from_user.id) + '}/' + name_obj 
        photo = open('pict.jpg', 'rb')
        bot.send_photo(message.from_user.id, photo)
    





def photo_name_handler(message):
    global photo_name
    photo_name = message.text
    bot.send_message(message.from_user.id, 'Теперь загрузи фото')
    bot.register_next_step_handler(message, photo_handler)

def photo_handler(message):
    if photo_name != None :
        if os.path.exists('PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}/' + photo_name + '.jpg'):
            bot.send_message(message.from_user.id, 'Картинка с таким именем уже есть')
            default_photo_handler(message)
        elif message.text != '':
            if os.path.exists('PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}'):
                directory = 'PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}'
            else:
                os.mkdir('PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}')
                directory = 'PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}'

            if message.photo:
                photo = message.photo[-1]
                file_id = photo.file_id
                file_path = bot.get_file(file_id).file_path
                downloaded_file = bot.download_file(file_path)
                name = photo_name + ".jpg"
                new_file = open(directory + '/' + name, mode='wb')
                new_file.write(downloaded_file)
                new_file.close()
                bot.send_message(message.from_user.id, 'Принял photo_handler')
    else :
        bot.send_message(message.from_user.id, 'Просила же имя для пикчи, теперь она абракадаброй называется')
        default_photo_handler(message)





@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Я догадываюсь, что тебя подослала Организация, чтобы выведать у меня секреты Хооина Кёмы")
    bot.send_message(message.from_user.id, text = 'Выбирай', reply_markup=markup)

@bot.message_handler(commands=['help'])
def send_welcome(message):
    bot.reply_to(message, 'Эль псай конгру' + '\n'+'Вот что я умею:' + '\n' + 'anime - рандомная аниме пикча'+ '\n' + 'webm - выбор видоса из общего хранилища' +
                 '\ncount - сколько сообщений ты мне прислал' + '\nlist - твои последние сообщения')
    bot.send_message(message.from_user.id, text = 'Выбирай', reply_markup=markup)

@bot.message_handler(content_types=['voice'])
def handler(message):
    bot.send_message(message.chat.id, 'Напиши словами, не могу послушать')


@bot.message_handler(content_types=['photo'])
def default_photo_handler(message):
    if os.path.exists( 'images_from_users/{' + str(message.from_user.id) + '}'):
        directory = 'images_from_users/{' + str(message.from_user.id) + '}'
    else:
        os.mkdir( 'images_from_users/{' + str(message.from_user.id) + '}')
        directory = 'images_from_users/{' + str(message.from_user.id) + '}'
    if message.photo:
        photo = message.photo[-1]
        file_id = photo.file_id
        file_path = bot.get_file(file_id).file_path
        downloaded_file = bot.download_file(file_path)
        name = file_id + ".jpg"
        new_file = open(directory + '/' + name, mode='wb')
        new_file.write(downloaded_file)
        new_file.close()
        bot.send_message(message.from_user.id, 'Принял default photo')




@bot.message_handler(func=lambda m: True)
def echo_all(message):
    if message.text:
        add_message(user_id = message.from_user.id, text = message.text)
        #bot.send_message(message.from_user.id, text = 'Выбирай', reply_markup=markup)

    if message.text == 'привет' :
        bot.reply_to(message, path_style)
    elif message.text == 'Srbija' :
        bot.reply_to(message,'strong')
    elif message.text == 'слава роду' :
        bot.reply_to(message,'нет уроду')
    elif message.text == 'если есть на свете рай' :
        bot.reply_to(message, 'это - Краснодарский край')
    elif message.text == 'rand pic' :
        pic_list = glob("PATH_SAVED_PICS{" + 'str(message.from_user.id)' + '}'+ '/*')
        picture = choice(pic_list)
        print(picture)
        photo = open(picture, 'rb')
        bot.send_photo(message.from_user.id, photo)
    elif message.text == 'pic' :
        pic_list = glob (PATH_GEN_PICS + '/*')
        textp = ''
        photo_num = 1
        for pic_name in pic_list :
            pic_name = pic_name[21:-4]
            textp = textp + ' ' + pic_name + ' - ' + str(photo_num) + '\n'
            photo_num += 1
        bot.send_message(message.from_user.id, 'Выбери номер фото из списка:\n' + textp)
        bot.register_next_step_handler(message, pic)
    
    elif message.text == 'my pic' :
        path = 'PATH_SAVED_PICS' + '/{' + str(message.from_user.id) + '}'
        if os.path.exists(path) :
            pic_list = glob(path + '/*')
            textp = ''
            photo_num = 1
            for pic_name in pic_list:
                pic_name = pic_name[53:-4]
                textp = textp + ' ' + pic_name + ' - ' + str(photo_num) + '\n'
                photo_num += 1
            bot.send_message(message.from_user.id, 'Выбери номер фото из списка:\n' + textp)
            bot.register_next_step_handler(message, my_pic)
        else :
            bot.send_message(message.from_user.id, 'Ты еще не отправлял мне фото')
    elif message.text == 'save pic' :
        bot.send_message(message.from_user.id, 'Придумай название для фото')
        bot.register_next_step_handler(message, photo_name_handler)
    
    if message.text == 'count' :
        nummes = str(count_message(user_id = message.from_user.id))
        bot.send_message(message.from_user.id, text = 'Столько ты отправил сообщений: ' + nummes[1:-2])
    if message.text == 'list' :
        nummes = int(str(count_message(user_id = message.from_user.id))[1:-2])
        nummes = min(nummes, 10)
        listmes = list_message(user_id = message.from_user.id, limit = nummes)
        stri = ''
        i = 0
        for mes in listmes:
            if i != 0:
                stri = stri + str(mes)[1:-2] + '\n'
            i = 1

        bot.send_message(message.from_user.id, 'Твои сообщения начиная с последнего:\n' + stri)
    if message.text == key_name :
        bot.send_message(message.from_user.id, 'Пришли стиль для фото')
        bot.register_next_step_handler(message, photo_obj)
    '''if message.text:
        #add_message(user_id = message.from_user.id, text = message.text)
        bot.send_message(message.from_user.id, text = 'Выбирай', reply_markup=markup)
    '''


#bot.get_me()
bot.polling()